<font color=lightblue size=20>Projeto Telecom X: Análise de evasão de clientes

# 📌 Extração

- **Importação:** Optei por usar a url do arquivo raw do GitHub a fim de facilitar a utilização do notebook por um usuário com poucos conhecimentos de importação de dados, visto que dessa forma seria necessário apenas executar o código ao invés de precisar importar o arquivo com os dados. Para tal fim foi necessário usar a biblioteca `requests` para buscar os dados e usar `json.loads()` para carregar os dados corretamente.

- **Normalização:** Como haviam colunas aninhadas (customer, phone, internet, account) utilizei `json_normalize()` para expandir as colunas.

## Código

In [1]:
import pandas as pd
import numpy as np
import requests
import json

url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'

response = requests.get(url) # Requests usado para carregar os dados json
response.raise_for_status() # Lança uma exceção para códigos de status HTTP ruins (4xx ou 5xx)

# Carrega o conteúdo JSON como um objeto Python
dados_json = json.loads(response.text)

# Normaliza os dados
df_normalizado = pd.json_normalize(dados_json, sep='_')

## Exibição dos dados

In [2]:
df_normalizado.head()

,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,...,internet_OnlineBackup,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges_Monthly,account_Charges_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


#🔧 Transformação

- **Duplicados:** Não encontrei dados duplicados.

- **nan e null:** Não encontrei nos dados.

- **Conversão de tipos:** Ao ferificar os tipos de dados percebi que a coluna com os gastos totais de cada cliente estava em `object`, por isso fiz a conversão para `np.float64`. Para isso precisei alterar strings vazias para 0.

- **Strings:** Padronizei os textos para letras minúsculas. Para isso criei uma variável somente com os nomes das colunas com texto e iterei sobre `df` utilizando essa variável como referência transformando em minúscula coluna por coluna.

- **Colunas binárias:** Nas colunas que apresentavam apenas dois valores de sim e não transformei sim em 1 e não em 0.

- **Tradução:** Os nomes das colunas bem como os dados em inglês foram traduzidos para o português a fim de facilitar a vizualização e entendimento dos dados.

- **Simplicação:** Na coluna `multiplas_linhas` o termo 'No phone service' foi simplificado para 'na' de não se aplica. O mesmo foi feito nas colunas `protecao_online, backup_online, protecao_dispositivo, suporte_tecnico, tv_streaming, filmes_streaming:` com o termo 'No internet service'.

- **Remoção da coluna `id`:** Reemovi a coluna `id` pois não é relevante para a análise.

- **Coluna `Contas_Diarias`:** Criação da coluna `Contas_Diarias`, dividindo a coluna `cobranca_mensal` por 30 dias, para uma visão mais detalhada do comportamento dos clientes ao longo do tempo.

- **round(2):** Deixei as colunas com tipo float com duas casas decimais.

- **Dicionário de dados:** No final deste notebook vai encontrar um dicionário de dados com uma breve explicação das informações contidas em cada coluna.

## Código

In [3]:
# ANALISANDO OS DADOS
print(f'TOTAL DE DUPLICADOS: {df_normalizado.duplicated().sum()}') # Conferindo duplicados
for item in df_normalizado.columns: # verificando as colunas
   print(f'{item:>30}: {df_normalizado[item].dtype} => NA: {df_normalizado[item].isna().sum()} '
   f'=> NULL: {df_normalizado[item].isnull().sum()} => DADOS: {pd.unique(df_normalizado[item])}')

TOTAL DE DUPLICADOS: 0
                    customerID: object => NA: 0 => NULL: 0 => DADOS: ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
                         Churn: object => NA: 0 => NULL: 0 => DADOS: ['No' 'Yes' '']
               customer_gender: object => NA: 0 => NULL: 0 => DADOS: ['Female' 'Male']
        customer_SeniorCitizen: int64 => NA: 0 => NULL: 0 => DADOS: [0 1]
              customer_Partner: object => NA: 0 => NULL: 0 => DADOS: ['Yes' 'No']
           customer_Dependents: object => NA: 0 => NULL: 0 => DADOS: ['Yes' 'No']
               customer_tenure: int64 => NA: 0 => NULL: 0 => DADOS: [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
            phone_PhoneService: object => NA: 0 => NULL: 0 => DADOS: ['Yes' 'No']
           phone_MultipleLines: object 

In [4]:
# ALTERANDO COLUNA DE GASTOS TOTAIS PARA FLOAT
# Substituindo strings vazias por NAN
df_normalizado['account_Charges_Total'] = df_normalizado['account_Charges_Total'].replace(' ', 0)
# Alterando o tipo de dados da coluna de str para float
df_normalizado['account_Charges_Total'] = df_normalizado['account_Charges_Total'].astype(np.float64)

In [5]:
# MODIFICANDO COLUNAS COM STRINGS PARA LOWER CAPS
# Separando colunas com string
colunas_tratar = df_normalizado.columns
colunas_tratar = colunas_tratar.drop(['account_Charges_Monthly', 'account_Charges_Total',
                        'customer_SeniorCitizen', 'customer_tenure'])
# aplicando lower no nas colunas com string
for column in df_normalizado.columns:
  if column in colunas_tratar:
    df_normalizado[column] = df_normalizado[column].str.lower()

In [6]:
# CONVERTENDO STRINGS DE YES E NO EM 1 E 0
# Substituindo strings vazias por no na coluna churn
df_normalizado['Churn'] = df_normalizado['Churn'].replace(r'^\s*$', 'no', regex=True)
# Pegando colunas com apenas yes e no
colunas_converter = ['Churn', 'customer_Partner', 'customer_Dependents',
                  'phone_PhoneService', 'account_PaperlessBilling']
# Substituindo yes e no por 1 e 0
for column in df_normalizado.columns:
  if column in colunas_converter:
    df_normalizado[column] = df_normalizado[column].map({'yes': 1, 'no':0})

In [7]:
# RENOMEANDO AS COLUNAS PARA PT-BR
# Criar um dicionário de mapeamento para renomear as colunas
novos_nomes_colunas = {
    'customerID': 'id_cliente',
    'Churn': 'evasao',
    'customer_gender': 'genero',
    'customer_SeniorCitizen': 'idoso',
    'customer_Partner': 'parceiro',
    'customer_Dependents': 'dependentes',
    'customer_tenure': 'meses_contrato',
    'phone_PhoneService': 'servico_telefone',
    'phone_MultipleLines': 'multiplas_linhas',
    'internet_InternetService': 'servico_internet',
    'internet_OnlineSecurity': 'protecao_online',
    'internet_OnlineBackup': 'backup_online',
    'internet_DeviceProtection': 'protecao_dispositivo',
    'internet_TechSupport': 'suporte_tecnico',
    'internet_StreamingTV': 'tv_streaming',
    'internet_StreamingMovies': 'filmes_streaming',
    'account_Contract': 'tipo_contrato',
    'account_PaperlessBilling': 'fatura_digital',
    'account_PaymentMethod': 'metodo_pagamento',
    'account_Charges_Monthly': 'cobranca_mensal',
    'account_Charges_Total': 'cobranca_total'
}
# Recomenado as colunas
df_traduzido = df_normalizado.rename(columns=novos_nomes_colunas)

In [8]:
# TRADUZINDO TERMOS DO EN-US PARA PT-BR
# Colunas para traduzir
colunas_traduzir = [ 'genero', 'multiplas_linhas', 'servico_internet',
                    'protecao_online', 'backup_online', 'protecao_dispositivo',
                     'suporte_tecnico', 'tv_streaming', 'filmes_streaming',
                     'tipo_contrato', 'metodo_pagamento']
# Dicionário com as palavras para traduzir
traducao = {'female': 'mulher', 'male': 'homem',
            'no': 'nao',
            'yes': 'sim',
            'fiber optic': 'fibra otica',
            'no phone service': 'na',
            'no internet service': 'na',
            'one year': 'anual',
            'month-to-month':'cada mes',
            'two year': 'bienal',
            'mailed check': 'cheque correios',
            'electronic check': 'cheque eletronico',
            'credit card (automatic)': 'cartao credito automatico',
            'bank transfer (automatic)': 'transferencia automatica'}
# executando a traduçao nas colunas
for column in df_traduzido.columns:
  if column in colunas_traduzir:
    df_traduzido[column] = df_traduzido[column].replace(traducao)

In [9]:
# DROPANDO A COLUNA ID
df_clientes = df_traduzido.drop(columns='id_cliente')

In [10]:
# ARREDONDANDO A COLIUNA cobranca_mensal PARA 2 CASAS DECIMAIS
df_clientes['cobranca_mensal'] = df_clientes['cobranca_mensal'].round(2)

In [11]:
# CRIANDO A COLUNA Contas_Diarias
df_clientes['contas_diarias'] = (df_clientes['cobranca_mensal'] / 30).round(2)

In [12]:
# CONFERINDO OS DADOS
print(f'TOTAL DE DUPLICADOS: {df_clientes.duplicated().sum()}') # Conferindo duplicados
for item in df_clientes.columns: # verificando as colunas
   print(f'{item:>30}: {df_clientes[item].dtype} => NA: {df_clientes[item].isna().sum()} '
   f'=> NULL: {df_clientes[item].isnull().sum()} => DADOS: {pd.unique(df_clientes[item])}')

TOTAL DE DUPLICADOS: 179
                        evasao: int64 => NA: 0 => NULL: 0 => DADOS: [0 1]
                        genero: object => NA: 0 => NULL: 0 => DADOS: ['mulher' 'homem']
                         idoso: int64 => NA: 0 => NULL: 0 => DADOS: [0 1]
                      parceiro: int64 => NA: 0 => NULL: 0 => DADOS: [1 0]
                   dependentes: int64 => NA: 0 => NULL: 0 => DADOS: [1 0]
                meses_contrato: int64 => NA: 0 => NULL: 0 => DADOS: [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
              servico_telefone: int64 => NA: 0 => NULL: 0 => DADOS: [1 0]
              multiplas_linhas: object => NA: 0 => NULL: 0 => DADOS: ['nao' 'sim' 'na']
              servico_internet: object => NA: 0 => NULL: 0 => DADOS: ['dsl' 'fibra otica' 'nao']
               protecao_online: object =

## Exibição dos dados

In [13]:
df_clientes.head()

,evasao,genero,idoso,parceiro,dependentes,meses_contrato,servico_telefone,multiplas_linhas,servico_internet,protecao_online,...,protecao_dispositivo,suporte_tecnico,tv_streaming,filmes_streaming,tipo_contrato,fatura_digital,metodo_pagamento,cobranca_mensal,cobranca_total,contas_diarias
0,0,mulher,0,1,1,9,1,nao,dsl,nao,...,nao,sim,sim,nao,anual,1,cheque correios,65.6,593.30,2.19
1,0,homem,0,0,0,9,1,sim,dsl,nao,...,nao,nao,nao,sim,cada mes,0,cheque correios,59.9,542.40,2.00
2,1,homem,0,0,0,4,1,nao,fibra otica,nao,...,sim,nao,nao,nao,cada mes,1,cheque eletronico,73.9,280.85,2.46
3,1,homem,1,1,0,13,1,nao,fibra otica,nao,...,sim,nao,sim,sim,cada mes,1,cheque eletronico,98.0,1237.85,3.27
4,1,mulher,1,1,0,3,1,nao,fibra otica,nao,...,nao,sim,sim,nao,cada mes,1,cheque correios,83.9,267.40,2.80


#📊 Carga e análise

## Análise descritiva dos dados

### Código

In [16]:
df_clientes.describe()

,evasao,idoso,parceiro,dependentes,meses_contrato,servico_telefone,fatura_digital,cobranca_mensal,cobranca_total,contas_diarias
count,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000
mean,0.257190,0.162653,0.484106,0.300124,32.346498,0.902711,0.593230,64.720098,2277.182035,2.157292
std,0.437115,0.369074,0.499782,0.458343,24.571773,0.296371,0.491265,30.129572,2268.648587,1.004407
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,0.000000,0.610000
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,35.425000,396.200000,1.180000
50%,0.000000,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,70.300000,1389.200000,2.340000
75%,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,89.875000,3778.525000,2.995000
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,118.750000,8684.800000,3.960000


### Desvio padrão

- Nas colunas `meses_contrato` e `cobranca_mensal` encontrei um desvio padrão grande. Indica uma alta variabilidade na variável correspondente. Significa que há uma grande diferença entre os clientes que gastam pouco e os que gastam muito nessa métrica.

- A coluna `cobranca_total` apresenta um desvio padrão extremamente alto. Isso indica que há uma dispersão massiva dos dados. Um desvio padrão tão alto aqui significa que a diferença entre o cliente que menos gastou e o que mais gastou é gigantesca.

### Médias e medianas

Avaliando as medianas e a média de cada coluna descobri uma diferença muito alta na coluna `cobranca_total`. isso indica que pode haver um pequeno grupo de clientes com gastos excepcionalmente altos puxando a média para cima, enquanto a maioria gasta menos.

## Exibição dos dados

In [15]:
df_clientes.describe()

,evasao,idoso,parceiro,dependentes,meses_contrato,servico_telefone,fatura_digital,cobranca_mensal,cobranca_total,contas_diarias
count,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000
mean,0.257190,0.162653,0.484106,0.300124,32.346498,0.902711,0.593230,64.720098,2277.182035,2.157292
std,0.437115,0.369074,0.499782,0.458343,24.571773,0.296371,0.491265,30.129572,2268.648587,1.004407
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,0.000000,0.610000
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,35.425000,396.200000,1.180000
50%,0.000000,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,70.300000,1389.200000,2.340000
75%,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,89.875000,3778.525000,2.995000
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,118.750000,8684.800000,3.960000


## Distribuição da Evasão

### Código

In [45]:
import matplotlib.pyplot as plt
import plotly.express as px

evasao_proporcao = {'Evasão': ['Sem Evasão', 'Evasão'],
                    'Proporção' : df_clientes['evasao'].value_counts(normalize=True)}

def grafico_evasao_proporcao():
  fig = px.pie(evasao_proporcao,
               values='Proporção',
               names='Evasão',
               title='Proporção de Clientes com e sem Evasão',
               labels={'Evasão': 'Status', 'Proporção': 'Proporção de Clientes'},
               color_discrete_sequence=px.colors.qualitative.Pastel,
               width=1000,
               height=600)
  fig.show()

### Proporção de clientes com evasão

Uma análise da distribuição da evasão ajudou a identificar uma taxa de aproximadamente 25% de evasão dos cleintes.

In [46]:
grafico_evasao_proporcao()

#📄Relatorio Final

# Relatório de Análise de Evasão de Clientes (Churn) - Projeto Telecom X

## 1. Introdução

Este relatório detalha a análise de evasão de clientes, ou "Churn", no contexto do Projeto Telecom X. O principal objetivo desta análise é compreender os fatores que levam os clientes a cancelar seus serviços, ou seja, a "evadir" da empresa. A evasão de clientes é um desafio significativo para as empresas de telecomunicações, pois impacta diretamente a receita e a sustentabilidade do negócio. Ao identificar os padrões e características dos clientes que evadem, a empresa pode desenvolver estratégias mais eficazes para retenção, como ofertas personalizadas, melhoria do atendimento ao cliente ou ajuste de planos e serviços. Em última análise, a meta é reduzir a taxa de evasão e, consequentemente, otimizar a base de clientes.

## 2. Limpeza e Tratamento de Dados

A etapa de limpeza e tratamento de dados foi fundamental para garantir a qualidade e a consistência das informações, preparando-as para as análises subsequentes. Os passos realizados foram os seguintes:

* **Importação de Dados:** Os dados foram importados a partir de um arquivo JSON hospedado no GitHub. Para facilitar a utilização e evitar a necessidade de importação manual do arquivo, foi utilizada a biblioteca `requests` para buscar o conteúdo diretamente da URL (`https://raw.githubusercontent.com/digitalinnovationone/dio-bootcamp-ciencia-de-dados-telecom/main/challenge-telecom.json`), e `json.loads()` para carregar os dados em formato JSON.

* **Normalização de Colunas Aninhadas:** O conjunto de dados original continha colunas aninhadas (como `customer`, `phone`, `internet` e `account`). Para expandir essas estruturas e transformá-las em colunas independentes no DataFrame principal, foi utilizada a função `pd.json_normalize()`.

* **Renomeação de Colunas:** As colunas foram renomeadas para um formato mais legível e padronizado, facilitando a manipulação e compreensão dos dados. Exemplos incluem `customer.customer_id` para `id_cliente`, `churn.churn` para `evasao`, entre outros.

* **Tratamento da Coluna `cobranca_total`:**
    * Identificou-se que a coluna `cobranca_total` continha valores em branco (`' '`) que impediam sua conversão direta para um tipo numérico. Esses espaços foram substituídos por valores `NaN` (Not a Number) para serem tratados posteriormente.
    * Após a substituição, a coluna foi convertida para o tipo numérico (float).
    * Os valores `NaN` resultantes da conversão foram preenchidos com `0`, assumindo que valores em branco nessa coluna representam ausência de cobrança total ou um valor zero para clientes novos.

* **Mapeamento da Coluna `evasao`:** A coluna `evasao`, que indicava a saída do cliente como 'Yes' ou 'No', foi convertida para um formato numérico binário para facilitar a análise e modelagem. 'Yes' foi mapeado para `1` (indicando evasão) e 'No' para `0` (indicando não evasão).

Esses passos garantiram que o DataFrame estivesse limpo, estruturado corretamente e pronto para a exploração de dados e a construção de modelos preditivos de evasão.


***
Análise Exploratória de Dados: Apresente as análises feitas, incluindo gráficos e visualizações para identificar padrões.

Conclusões e Insights: Resuma os principais achados e como esses dados podem ajudar a reduzir a evasão.

Recomendações: Ofereça sugestões baseadas na sua análise.

Certifique-se de que o relatório esteja bem estruturado, claro e com as visualizações que sustentam suas conclusões.

# **Dicionário de dados**



* **id_cliente**: número de identificação único de cada cliente (String)
* **evasao**: se o cliente deixou ou não a empresa. (int64 / 0 = não e 1 = sim)
* **genero**: gênero (masculino e feminino)
* **senior_citizen**: informação sobre um cliente ter ou não idade igual ou maior que 65 anos
* **parceiro**: se o cliente possui ou não um parceiro ou parceira
* **dependentes**: se o cliente possui ou não dependentes
* **meses_contrato**: meses de contrato do cliente
* **servico_telefone**: assinatura de serviço telefônico
* **multiplas_linhas**: assinatura de mais de uma linha de telefone (String / 'nao', 'sim' e 'na' para não se aplica)
* **servico_internet**: assinatura de um provedor internet
* **backup_online**: assinatura adicional de backup online
* **protecao_dispositivo**: assinatura adicional de proteção no dispositivo
* **suporte_tecnico**: assinatura adicional de suporte técnico, menos tempo de espera
* **tv_streaming**: assinatura de TV a cabo
* **filmes_streaming**: assinatura de streaming de filmes
* **tipo_contrato**: tipo de contrato
* **fatura_digital**: se o cliente prefere receber online a fatura
* **metodo_pagamento**: forma de pagamento
* **cobranca_mensal**: total de todos os serviços do cliente por mês
* **cobranca_total**: total gasto pelo cliente